In [1]:
#Imports
import json
import numpy as np
import pandas as pd
import csv
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open(path,'r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [3]:
def slicesParser_amplitudes(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['amplitudes']

    return df_sum

In [ ]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/'
experiments = os.listdir(path + 'jsons/')
dif_amp = pd.DataFrame()
for i in range(0,len (experiments)):
    experiment = jasonReader(path + 'jsons/' + experiments[i]) 
    try:
        amp_file = slicesParser_amplitudes(experiment)
    except:
        print(experiments[i])
    try:
        dif_amp [i]= experiments[i],(amp_file['transient1'] - amp_file['transient2']).mean()
    except:
        pass
dif_amp = pd.DataFrame (dif_amp.T)
dif_amp.columns = ['file','dif_amp_mean']

### DICRI

In [70]:
tab = pd.read_csv('/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/seleccion/tabulado_imagenes.csv', encoding='utf-8')
del tab['Unnamed: 0']
pd.set_option('display.max_rows', None)
controles = tab[tab['tratamiento'] == 'c']
controles = controles.reset_index(drop=True)

In [73]:
exps = os.listdir(path + 'jsons/')
experiments = []
for j in range(0,len(exps)):
    for i in range(0,len(controles)):
        if str(controles['experimento'][i]) in exps[j]:
            if str(controles['foto'][i]) in exps[j]:
                experiments.append(exps[j])

In [76]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/'
# experiments = os.listdir(path + 'jsons/')
complete_file = pd.DataFrame()
for i in range(0,len (experiments)):
    try:
        experiment = jasonReader(path + 'jsons/' + experiments[i])
        amp_files = slicesParser_amplitudes(experiment).filter(['transient1','transient2'])
        amp_files = amp_files[:][0:33]
        transient_max = []
        for j in range(1,len(amp_files)+1):
            transient_max.append(max(amp_files['transient1'][j],amp_files['transient2'][j]))
        amp_files['transient_max'] = transient_max
        amp_files.loc[len(amp_files)+1] = (amp_files.mean())
        amp_files.loc[len(amp_files)+1] = [amp_files['transient1'][len(amp_files)]*100/amp_files['transient_max'][len(amp_files)],amp_files['transient2'][len(amp_files)]*100/amp_files['transient_max'][len(amp_files)],'']
        amp_files[str(experiments[i])] = list(range(1,len(amp_files)-1))+['promedios']+['porcentaje_max']
        complete_file = pd.concat([complete_file,amp_files], axis=1)
    except ValueError:
        print(experiments[i])

<ipython-input-76-6d31658230a6>:14: RuntimeWarning: invalid value encountered in double_scalars
  amp_files.loc[len(amp_files)+1] = [amp_files['transient1'][len(amp_files)]*100/amp_files['transient_max'][len(amp_files)],amp_files['transient2'][len(amp_files)]*100/amp_files['transient_max'][len(amp_files)],'']


060121_analysis_result_c9d000.json


In [77]:
complete_file.to_excel(path + 'DICRI_controles.xlsx', engine='xlsxwriter')  

### Amplitud de transitorio global vs DI

In [ ]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/json_nuevos/'
file_tabulado = pd.read_csv(path + 'tabulado_parceado.csv')
file_tabulado.head() 

In [ ]:
experiment = jasonReader(path + 'jsons/050320c8_analysis_result_8b000.json')
allSlices = experiment['slices']
for i in range(0,len(allSlices)):
    print(allSlices[i]['amplitudes'])

In [ ]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/json_nuevos/whole_cell/'
experiments = os.listdir(path)
df = pd.DataFrame()
for experiment in experiments:
    df = pd.read_csv(path + experiment, sep = '\t', index_col=0).loc['amplitudes']
    for imagen in range(0,len(file_tabulado)):
        cel = file_tabulado['célula'].loc[imagen]
        foto = file_tabulado['foto'].loc[imagen]
        if (cel in experiment) & (foto in experiment):
            print(experiment,df)

#     print (experiment, df)

### Precisión y exactitud amplitud

In [4]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
file_tabulado = pd.read_excel(path + 'tab_prueba2.xlsx')
del file_tabulado['Unnamed: 0']

In [5]:
df_manual = pd.read_csv("/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/DI_manual.csv", sep=",")

In [9]:
df_manual.head(2)

,experimento,foto,pico,Intensidad_max,tiempo_max,Intensidad_min,tiempo_min,Amplitud,TTP
0,200121,C1a000,1,43.19,125,17.89,118,1.414198,7
1,200121,C1a000,2,42.64,360,16.80,354,1.538095,6


In [10]:
file_tabulado.head(2)

,experimento,carpeta,foto,tratamiento,frecuencia,dosis,DI
0,31019,031019c2,c2c000,vk,3,5,0.246228
1,31019,031019c2,c2d000,vk,4,5,NaN


In [15]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/'
experiments = os.listdir(path + 'jsons/')
dif_amp = pd.DataFrame()
for i in range(0,len (experiments)):
    
#     try:
#         experiment = jasonReader(path + 'jsons/' + experiments[i]) 
#         amplitud = slicesParser_amplitudes(experiment)
#         print(amplitud)
#     except:
#         print(experiments[i])

    transient1  transient2  transient3  transient4  transient5  transient6
1     1.692308    1.500000    1.500000    1.615385    1.538462    1.692308
2     1.833333    1.615385    1.615385    1.833333    1.666667    1.615385
3     1.750000    1.833333    1.538462    1.750000    1.583333    1.750000
4     1.750000    1.750000    1.461538    1.666667    1.583333    1.750000
5     1.538462    1.833333    1.538462    1.461538    1.750000    1.750000
6     1.500000    1.692308    1.500000    1.357143    1.428571    1.615385
7     1.400000    1.642857    1.400000    1.333333    1.571429    1.333333
8     1.250000    1.466667    1.400000    1.250000    1.466667    1.187500
9     1.250000    1.466667    1.400000    1.250000    1.250000    1.187500
10    1.250000    1.466667    1.250000    1.250000    1.250000    1.187500
11    1.375000    1.250000    1.312500    1.250000    1.312500    1.250000
12    1.294118    1.312500    1.176471    1.312500    1.312500    1.312500
13    1.600000    1.46666

In [58]:
for i in range(0,len(file_tabulado['carpeta'])):
    cel1 = file_tabulado['carpeta'][i]
    for j in range(0,len(df_manual['cel'])):
        cel2 = df_manual['cel'][j]
        try:
            if cel2 in cel1:
                 file_tabulado.loc[i,'DI_manual'] = df_manual['DI_manual'][j]
        except TypeError:
            pass

In [59]:
file_DIs = file_tabulado.dropna()
file_DIs[(file_DIs['frecuencia']==1) & (file_DIs['tratamiento']=='c')]

,experimento,carpeta,foto,tratamiento,frecuencia,dosis,DI,DI_manual
16,31019,031019c1,c1b000,c,1,0,0.120523,0.465901
99,121218,121218,c1a000,c,1,0,0.130269,0.116015
176,10719,010719ctrol1,ctrol1b0,c,1,0,0.683346,0.523415
186,10719,010719ctrol8,ctrol8b0,c,1,0,0.100734,0.182409
337,230819,230819c1,otra1hz1,c,1,0,0.050558,0.135194
354,10819,010819c1,c1a000,c,1,0,0.260349,0.300057
366,171019,171019c1sel,c1a000,c,1,0,0.124389,0.134509
377,171019,171019c6sel,c6a000,c,1,0,0.048430,0.154635


### ALBUMINA

In [ ]:
# Python code demonstrate creating  
# DataFrame from dict narray / lists  
# By default addresses. 
  
import pandas as pd 
  
# intialise data of lists. 
data = {'BSA':[102, 97, 79], 
        'HSA':[65, 74, 69.5],
        'RabSA':[57, 63.3, 70],
        'RSA':[50, 51, 52],
        'PSA':[45, 48, 55]} 
  
# Create DataFrame 
df = pd.DataFrame(data)
df_mean = df.mean()
df_std = df.std()
# Print the output. 
df_std

In [ ]:
import matplotlib.pyplot as plt

plt.errorbar(df_mean.index, df_mean, yerr=df_std, xerr=None, fmt='o')
plt.ylabel('% convertion', fontdict=None, labelpad=None,loc=None)
plt.savefig('/media/leandro/Volumen1TB/Lean/Albumina/%alb_convertion.png')

In [ ]:
cambios_fq = pd.read_csv('/media/leandro/Volumen1TB/Lean/Albumina/cambios_fq_completos.csv', sep= ' ')

In [ ]:
cambios_fq.loc()